# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [58]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import time

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17878428870996721110
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10507485184
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17932028037471028523
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:05:00.0, compute capability: 6.1"
]


## Env setup

In [44]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [45]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [46]:
# What model to download.
# MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_NAME = 'ssd_mobilenet_v1_coco_2018_01_28'
# MODEL_NAME = 'ssd_mobilenet_v1_ppn_shared_box_predictor_300x300_coco14_sync_2018_07_03'
#MODEL_NAME = 'faster_rcnn_inception_v2_coco_2018_01_28'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

## Download Model

In [47]:
opener = urllib.request.URLopener()
# retrieve(URL, filename)
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# 圧縮されたフォルダの一覧
tar_file = tarfile.open(MODEL_FILE)
# 回答したフォルダの中からgraph.pdファイルを探す
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    # 解凍
    tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [48]:
detection_graph = tf.Graph()
print(PATH_TO_FROZEN_GRAPH)
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

ssd_mobilenet_v1_coco_2018_01_28/frozen_inference_graph.pb


## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [49]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper code

In [50]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

動画(.avi)を画像に変換

In [51]:
import cv2
WORK_DIR = os.environ["WORK_DIR"]
DATA_DIR = os.path.join(WORK_DIR,"data")
data = []
for i in os.listdir(DATA_DIR):
    data_list = os.path.join(DATA_DIR,i)
    for j in os.listdir(data_list):
        data.append(os.path.join(data_list,j))

## 以下は初回実行のみ
# for i in data:
#    count = 0
#    cap = cv2.VideoCapture(i)
#    filename = os.path.basename(i)
#    while True:
#        ret, frame = cap.read()
#        if ret == True:
#            count += 1
#            save_dir = os.path.join(os.path.dirname(i), os.path.splitext(filename)[0])
#            if not os.path.exists(save_dir):
#                os.mkdir(save_dir)
# 
#            cv2.imwrite(os.path.join(save_dir, "{0:05d}".format(count) + ".jpg"),frame)
#        else:
#             break

In [52]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      t1 = time.time()
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})
      t2 = time.time()
      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict, t2-t1

In [103]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
# PATH_TO_TEST_IMAGES_DIR = 'test_images'
# TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]
# ファイル数カウント

filename = os.path.basename(data[-1])
file_dir = os.path.dirname(data[-1])

image_dir = os.path.join(file_dir, os.path.splitext(filename)[0])

# TODO もっと簡略
import re
count = 0
for file in os.listdir(image_dir):
    index = re.search(".jpg", file)
    if index:
        count = count + 1
        
image_path_list = [os.path.join(image_dir, "{:05d}".format(i)) + ".jpg" for i in range(1,count+1)]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [110]:
def filter_boxes(min_score, output, categories):
    idxs = []
    n = len(output['detection_classes'])
    for i in range(n):
        if output['detection_classes'][i] in categories and output['detection_scores'][i] >= min_score:
            idxs.append(i)
            
    if len(idxs) == 0:
        raise Exception('No Detect Person!')
            
    return output['detection_boxes'][idxs],output['detection_scores'][idxs],output['detection_classes'][idxs]

def draw_rect(filtered_output, image_path, output_path):
    img = cv2.imread(image_path, 1)
    height = img.shape[0]
    width = img.shape[1]
    for rect in filtered_output:
        cv2.rectangle(img,(int(width*rect[1]), int(height*rect[0])),(int(width*rect[3]), int(height*rect[2])),(0,0,220),3)
    cv2.imwrite(output_path,img)

In [ ]:
result_dir = image_dir + "_result"
if not os.path.exists(result_dir):
    os.mkdir(result_dir)
    
run_time_total = 0

for i, image_path in enumerate(image_path_list):
  if i % 100 == 0:
    print("処理中:{}".format(i))
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict,run_time = run_inference_for_single_image(image_np, detection_graph)
  # print(output_dict)
  try:
    filtered_output = filter_boxes(0.2,output_dict, [1])
    draw_rect(filtered_output[0], image_path, os.path.join(result_dir, os.path.basename(image_path)))
  except:
    continue
  finally:
    run_time_total = run_time_total + run_time

  # Visualization of the results of a detection.
  # vis_util.visualize_boxes_and_labels_on_image_array(
  #     image_np,
  #     output_dict['detection_boxes'],
  #     output_dict['detection_classes'],
  #     output_dict['detection_scores'],
  #     category_index,
  #     instance_masks=output_dict.get('detection_masks'),
  #     use_normalized_coordinates=True,
  #     line_thickness=8)
  #plt.figure(figsize=IMAGE_SIZE)
  #plt.imshow(image_np)
  #plt.savefig(os.path.join(result_dir, os.path.basename(image_path)))

print(image_dir)
total_image_count = len(image_path_list)
print("処理枚数:{}, 単位実行時間:{}".format(total_image_count,(run_time_total / total_image_count)))

処理中:0
